In [ ]:
# Obtain Canvas access token from .env file

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())
token = os.environ.get("TOKEN")

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
PROVIDER = 'swinburne'

In [ ]:
def parse_iso_timestamps(data : pd.DataFrame) -> pd.DataFrame:
    """
    Replace all string timestamps in a DataFrame with datetime objects.
    All timestamps must comply with ISO-8601.

    Args:
        data (DataFrame): DataFrame containing ISO-8601 string timestamps.

    Returns:
        data (DataFrame): DataFrame containing datetime timestamps.
    """
    
    timestamp_columns = []
    
    for key, value in data.iloc[0].to_dict().items():
        
        try:
            datetime.fromisoformat(value)
            timestamp_columns.append(key)
        except:
            pass
    
    for column in timestamp_columns:
        data[column] = data[column].map(datetime.fromisoformat)

    return data
    
def decode_canvas_response(response : requests.Response) -> pd.DataFrame:
    """
    Converts raw Canvas API response into a DataFrame for downstream use,
    or raises Exception if the response is invalid.

    Args:
        response (Response): The raw API response obtained from requests.

    Returns:
        data (DataFrame): The response data.
    """
    
    if response.ok:
    
        # Canvas API will always respond in JSON format,
        # but we obtain the response in bytes, so it
        # must be decoded into a raw string and then
        # encoded into a JSON object.
        
        response = response.content.decode('utf-8')
        response = json.loads(response)

        response = pd.DataFrame(response)

        response = parse_iso_timestamps(response)
    
    else:
        response.raise_for_status()

    return response

In [ ]:
# Preview all units student is enrolled in

response = requests.get(f"https://{PROVIDER}.instructure.com/api/v1/courses",
                 params={"access_token":token})
response = decode_canvas_response(response)

response[["id", "course_code", "created_at"]]

In [ ]:
# Preview all the assignments for the student's first unit

first_unit_id = str(response.id[0])

response = r.get(f"https://{PROVIDER}.instructure.com/api/v1/courses/{first_unit_id}/assignments",
                 params={"access_token":token})
response = decode_canvas_response(response)

response[["name", "due_at"]]